# Under Which Cup?

I found [this](https://www.braingle.com/brainteasers/48900/under-which-cup.html) puzzle on [Braingle](https://www.braingle.com/).

> You decide to play a game with your friend where your friend places a coin under one of three cups. Your friend would then switch the positions of two of the cups several times so that the coin under one of the cups moves with the cup it is under. You would then select the cup that you think the coin is under. If you won, you would receive the coin, but if you lost, you would have to pay.

> As the game starts, you realise that you are really tired, and you don't focus very well on the moving of the cups. When your friend stops moving the cups and asks you where the coin is, you only remember a few things:

> He put the coin in the rightmost cup at the start.

> He switched two of the cups 3 times.

> The first time he switched two of the cups, the rightmost one was switched with another.

> The second time he switched two of the cups, the rightmost one was not touched.

> The third and last time he switched two of the cups, the rightmost one was switched with another.

> You don't want to end up paying your friend, so, using your head, you try to work out which cup is most likely to hold the coin, using the information you remember.

> Which cup is most likely to hold the coin?

## Analysis

This is a classic [cup shuffle game][1] in which the magician shifts the coin from one cup to another while shuffling them. Luckily, in this instance of the game, our friend didn't resort to trickery. Although, we still can't tell which cup holds the coin accurately as our memory failed us.

Hmm. So, let's start by listing what we know and how can we approach the problem:

* Since, we do not have complete information about our environment (game), this problem is an example of [partially observable environment][2]. 
* To deal with partial observability we make use of the notion of belief states. Fully observable environments use black boxes called states to keep track of changes happening in the environment. Belief states are the sets of all the possible states an environment can be in at a particular time.
* The states in this game can be one of the following:
      
      {Coin, Empty, Empty}, {Empty, Coin, Empty}, {Empty, Empty, Coin}

* For an environment which exhibit n different states, the belief state could be as big as 2^n. But since in this problem n equals 3, we face no issues related to the size and storage of the belief state. An example of belief state is:

      {{Coin, Empty, Empty}, {Empty, Empty, Coin}}
    
* We can also define a "move" as a deliberate action by an agent made which possibly alters the belief state. In this game, the moves are made by our friend and each move switches two cups, possibly moving the coin and hence changing the true state.
* Consider an example in which the actual state of the game is:

      {Coin, Empty, Empty}
    
  If a move is made that switches the 1st and the 2nd cup the new state will be:
  
      {Empty, Coin, Empty}
      
  Also, notice that a move made that switches the 1st and the 3rd cup does not change the state of the game.
* Normally, a belief state is a result of an "action" performed on the previous belief state. In this case, our belief state is also affected by all the different moves applied to previous belief. Hence, the belief state in this game can be thought of as a set of ordinary belief states. We can implement this with the use of the "List" data structure. It will help us to keep track of the likelihood with which a state can occur (Or the different ways in which state can occur).
* Since, the final answer will be based on the likelihood of the coin being under a certain cup, we'll also need to define a [probability model][3] for the game. We'll use the [discrete uniform distribution][4] as our probability model as all the states in the belief state for this game seem to be equally likely (there is no information given that says otherwise)


[1]: https://www.google.com/search?client=ubuntu&hs=l7S&channel=fs&ei=xhghW-_tK4W5swG8ybHgCQ&q=cup+shuffle&oq=cup+shuffle&gs_l=psy-ab.3..0i71k1l8.8262.8262.0.8483.1.1.0.0.0.0.0.0..0.0....0...1.1.64.psy-ab..1.0.0....0.ZAEO0ap0dbI
[2]: https://en.wikipedia.org/wiki/Partially_observable_system
[3]: http://www.stat.yale.edu/Courses/1997-98/101/probint.htm
[4]: https://en.wikipedia.org/wiki/Discrete_uniform_distribution

## Data Structures

This section will list all the data structures we'll be needing for the program.
* Coin: A str representing presence of coin in a cup in a State.
* Empty: A str representing absence of coin in a cup in a State.
* State: A str that is of type 'CEE' indicating presence and absence of coin in each cup.
* BeliefState: A list of possible States that we believe the environment is in.
* Move: An ordered pair of int that indicates the cups to swap.

In [1]:
from typing import Tuple, List
from itertools import permutations, chain, combinations
from functools import partial
from fractions import Fraction

State = Tuple[str, str, str] # A Tuple representing state of the world, e.g. ('Coin', 'Empty', 'Empty')
BeliefState = List[State] # A list of State that represents our belief
Move = Tuple[int, int] # An ordered pair of int that tells which cups to swap, e.g. {0, 2} swaps first and last cups
Probability = Fraction

Coin = 'Coin'
Empty = 'Empty'

## Functions

This sections defines all the basic functions we will need to operate on the defined data structures.

* all_states(): returns a list of all possible states
* all_belief_states(): returns a list of all possible belief states
* all_moves(): returns a list of all possible moves
* swap(state, move): takes a state as input and returns new state by applying the given move
* update(belief_state, move): takes a belief state and applies a move returning the updated belief_state. 

In [2]:
def all_states() -> [State]:
    """Lists out all possible states in the game"""
    return list(set(permutations((Coin, Empty, Empty))))

def all_belief_states() -> [BeliefState]:
    """List out all possible belief states in the game"""
    return list(set(i) for i in powerset(all_states()))

def all_moves() -> [Move]:
    """List out all possible moves that can be made by our friend"""
    return list(permutations((0, 1, 2), 2))

def swap(state: State, move: Move) -> State:
    """Returns a new state by applying the move to current state"""
    mutable_list = list(state)
    mutable_list[move[0]] = state[move[1]]
    mutable_list[move[1]] = state[move[0]]
    return tuple(mutable_list)

def update(belief_state: BeliefState, move: Move) -> BeliefState:
    """Returns an updated belief state after applying the move"""
    apply_move = partial(swap, move=move)
    return list(map(apply_move, belief_state))

def powerset(iterable):
    """
    https://docs.python.org/3.6/library/itertools.html#itertools-recipes
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

Let's check if these work correctly.

In [3]:
all_states()

[('Empty', 'Empty', 'Coin'),
 ('Empty', 'Coin', 'Empty'),
 ('Coin', 'Empty', 'Empty')]

In [4]:
all_belief_states()

[set(),
 {('Empty', 'Empty', 'Coin')},
 {('Empty', 'Coin', 'Empty')},
 {('Coin', 'Empty', 'Empty')},
 {('Empty', 'Coin', 'Empty'), ('Empty', 'Empty', 'Coin')},
 {('Coin', 'Empty', 'Empty'), ('Empty', 'Empty', 'Coin')},
 {('Coin', 'Empty', 'Empty'), ('Empty', 'Coin', 'Empty')},
 {('Coin', 'Empty', 'Empty'),
  ('Empty', 'Coin', 'Empty'),
  ('Empty', 'Empty', 'Coin')}]

In [5]:
all_moves()

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

In [6]:
old_state = ('Coin', 'Empty', 'Empty')
move = (0, 1)
swap(old_state, move)

('Empty', 'Coin', 'Empty')

In [7]:
belief = {('Coin', 'Empty', 'Empty'), ('Empty', 'Coin', 'Empty')}
move = (0, 2)
update(belief, move)

[('Empty', 'Coin', 'Empty'), ('Empty', 'Empty', 'Coin')]

Alright! The basic functions are working perfectly. Now, let's move on to the solving part.

## Solution

#### He put the coin in the rightmost cup at the start.

In [8]:
belief = {('Empty', 'Empty', 'Coin')}

#### The first time he switched two of the cups, the rightmost one was switched with another.

In [9]:
belief = update(belief, (2, 0)) + update(belief, (2, 1))

In [10]:
belief

[('Coin', 'Empty', 'Empty'), ('Empty', 'Coin', 'Empty')]

#### The second time he switched two of the cups, the rightmost one was not touched.

In [11]:
move = (0, 1)
belief = update(belief, move)

In [12]:
belief

[('Empty', 'Coin', 'Empty'), ('Coin', 'Empty', 'Empty')]

#### The third and last time he switched two of the cups, the rightmost one was switched with another.

In [13]:
belief = update(belief, (2, 0)) + update(belief, (2, 1))

In [14]:
belief

[('Empty', 'Coin', 'Empty'),
 ('Empty', 'Empty', 'Coin'),
 ('Empty', 'Empty', 'Coin'),
 ('Coin', 'Empty', 'Empty')]

So, in the end we see that the state where the coin is in the third cup is twice more likely to occur compared to other states.